# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [19]:
# Import the WeatherPy_database.csv file. 
weather_database = os.path.join("Weather_Database/WeatherPy_Database.csv")
# Display sample data
weather_database_df = pd.read_csv(weather_database)
weather_database_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,66.99,38,100,5.99,overcast clouds
1,1,Nuuk,GL,64.1835,-51.7216,33.76,71,100,15.57,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,64.38,89,4,3.18,clear sky
3,3,Hilo,US,19.7297,-155.0900,76.73,70,75,5.75,light rain
4,4,Chuy,UY,-33.6971,-53.4616,73.67,68,100,4.43,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [23]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [24]:
# Filter the weather_database_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = weather_database_df.loc[
    (weather_database_df["Max Temp"] <= max_temp) & (weather_database_df["Max Temp"] >= min_temp)
]


# Display sample data
preferred_cities_df.count()

City_ID                561
City                   561
Country                559
Lat                    561
Lng                    561
Max Temp               561
Humidity               561
Cloudiness             561
Wind Speed             561
Current Description    561
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [25]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()
clean_travel_cities_df = preferred_cities_df
# Display sample data
clean_travel_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,66.99,38,100,5.99,overcast clouds
1,1,Nuuk,GL,64.1835,-51.7216,33.76,71,100,15.57,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,64.38,89,4,3.18,clear sky
3,3,Hilo,US,19.7297,-155.0900,76.73,70,75,5.75,light rain
4,4,Chuy,UY,-33.6971,-53.4616,73.67,68,100,4.43,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [27]:
# Create a new column "Hotel Name".
clean_hotel_df["Hotel Name"] = ""

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.99,overcast clouds,-33.6500,115.3333,
1,Nuuk,GL,33.76,overcast clouds,64.1835,-51.7216,
2,Bredasdorp,ZA,64.38,clear sky,-34.5322,20.0403,
3,Hilo,US,76.73,light rain,19.7297,-155.0900,
4,Chuy,UY,73.67,overcast clouds,-33.6971,-53.4616,
6,Alta Floresta,BR,71.69,light rain,-9.8756,-56.0861,
7,Jamestown,US,29.79,clear sky,42.0970,-79.2353,
9,Port Elizabeth,ZA,68.31,few clouds,-33.9180,25.5701,
10,Andros Town,BS,76.96,clear sky,24.7000,-77.7667,
11,Puerto Ayora,EC,69.55,overcast clouds,-0.7393,-90.3518,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [28]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [29]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in clean_hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        clean_hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{clean_hotel_df.loc[index, 'City']} - nearest hotel: {clean_hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
clean_hotel_df.head(15)

Starting hotel search
Busselton - nearest hotel: No hotel found
Nuuk - nearest hotel: Nordbo
Bredasdorp - nearest hotel: Victoria Hotel
Hilo - nearest hotel: Dolphin Bay Hotel
Chuy - nearest hotel: Alerces
Alta Floresta - nearest hotel: Villas Hotel
Jamestown - nearest hotel: DoubleTree Jamestown
Port Elizabeth - nearest hotel: Waterford Hotel
Andros Town - nearest hotel: No hotel found
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Butaritari - nearest hotel: No hotel found
Sambava - nearest hotel: Royal Sava Hôtel
Nuevo Laredo - nearest hotel: Best Western Plus
Atuona - nearest hotel: Pearl Resort
Botwood - nearest hotel: No hotel found
Maceio - nearest hotel: Maceió Hostel e Pousada
Rikitea - nearest hotel: Chez Bianca & Benoit
Vila Velha - nearest hotel: Hotel Prainha
Saint-Philippe - nearest hotel: Le Baril
Punta Arenas - nearest hotel: Hotel Lacolet
Kahului - nearest hotel: Maui Seaside Hotel
Albany - nearest hotel: No hotel found
Kathmandu - nearest hotel: Ar

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,66.99,overcast clouds,-33.6500,115.3333,No hotel found
1,Nuuk,GL,33.76,overcast clouds,64.1835,-51.7216,Nordbo
2,Bredasdorp,ZA,64.38,clear sky,-34.5322,20.0403,Victoria Hotel
3,Hilo,US,76.73,light rain,19.7297,-155.0900,Dolphin Bay Hotel
4,Chuy,UY,73.67,overcast clouds,-33.6971,-53.4616,Alerces
6,Alta Floresta,BR,71.69,light rain,-9.8756,-56.0861,Villas Hotel
7,Jamestown,US,29.79,clear sky,42.0970,-79.2353,DoubleTree Jamestown
9,Port Elizabeth,ZA,68.31,few clouds,-33.9180,25.5701,Waterford Hotel
10,Andros Town,BS,76.96,clear sky,24.7000,-77.7667,No hotel found
11,Puerto Ayora,EC,69.55,overcast clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [30]:
# Drop the rows where there is no Hotel Name.
cleaner_hotels_df = clean_hotel_df[clean_hotel_df["Hotel Name"].str.contains("No hotel found")==False]

# Display sample data
cleaner_hotels_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nuuk,GL,33.76,overcast clouds,64.1835,-51.7216,Nordbo
2,Bredasdorp,ZA,64.38,clear sky,-34.5322,20.0403,Victoria Hotel
3,Hilo,US,76.73,light rain,19.7297,-155.0900,Dolphin Bay Hotel
4,Chuy,UY,73.67,overcast clouds,-33.6971,-53.4616,Alerces
6,Alta Floresta,BR,71.69,light rain,-9.8756,-56.0861,Villas Hotel
7,Jamestown,US,29.79,clear sky,42.0970,-79.2353,DoubleTree Jamestown
9,Port Elizabeth,ZA,68.31,few clouds,-33.9180,25.5701,Waterford Hotel
11,Puerto Ayora,EC,69.55,overcast clouds,-0.7393,-90.3518,Hostal La Mirada De Solitario George
13,Sambava,MG,73.76,clear sky,-14.2667,50.1667,Royal Sava Hôtel
14,Nuevo Laredo,MX,75.47,overcast clouds,27.4763,-99.5164,Best Western Plus


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [31]:
# Create the output File (CSV)
hotels_output_csv = 'Vacation_Search/WeatherPy_vacation.csv'

# Export the City_Data into a csv
cleaner_hotels_df.to_csv(hotels_output_csv, index_label="Hotel_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [33]:
# Configure the map plot
hotels_map = cleaner_hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    color = "City",
    hover_cols = ['Hotel Name', 'Country', 'Current Description']
)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)